# Пример генерации текста рекурентной сетью

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np
import random
import sys

Using TensorFlow backend.


## Загружаем исходные данные

In [3]:
text = open('./anna_karenina.txt').read().lower()
print('corpus length:', len(text))

corpus length: 1710408


## Векторизуем текстовый корпус

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), 
             dtype=np.bool)

y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 93
nb sequences: 570123
Vectorization...


In [6]:
y.shape

(570123, 93)

## Строим модель

In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

Build model...


## Возвращает индекс выбранного символа для вектора вероятностей

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-32) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Запускаем обучение

In [ ]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

## Предсказания на предобученной модели

In [10]:
model.load_weights('weights.hd5')

In [12]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "елеги. левин сосчитал телеги, остался до"
елеги. левин сосчитал телеги, остался долли было степан аркадьич, как он все степан аркадьич, под нем стал и после своей дела своими не понимает в себя старика степан аркадьич, не совершенно было до ни не стал степан аркадьич, под нее и простить в своими стал от нее в том стал при нее того отчаяния и в том, что он не видел в том, что он понимают ни не после своего приглисали и после нее стал стал при него и не понимая от нее стал и стал

----- diversity: 0.5
----- Generating with seed: "елеги. левин сосчитал телеги, остался до"
елеги. левин сосчитал телеги, остался добрования, что унинтре домой и принцвает, под себе. весело долженс в деревни большими всегда проговорил степан аркадьический развлека, что он видел, что он у нее стороном пред ней в простит в ответил от нее возвожул его, он возвожания кататься примерить от нее приговор своими все мужчина, как ты отдал это было мой приятеление при св